In [ ]:
import pickle as pkl
import numpy as np

X = pkl.load(open('./data/X_processed.pkl', 'rb'))
y = pkl.load(open('./data/y_processed.pkl', 'rb'))

print(X.shape, y.shape)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

training_size = 6000

X_train = X[:training_size]
y_train = y[:training_size]

X_test = X[training_size:]
y_test = y[training_size:]

X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

classifier = RandomForestClassifier()
y_pred = classifier.fit(X_train, y_train).predict(X_test)

accuracy_score(y_test, y_pred)

0.92629482071713143

In [ ]:
noise = np.random.normal(0, 1, 600)
plt.hist(noise, bins=30)
print(np.where(abs(noise)<0.01))

In [ ]:
#directory = './output/2.0_0.001_200_0.002_100/'
#directory = './output/4.0_0.0001_500_0.002_100/'
directory = './output/p9_8.0_0.0001_500_0.002_100/'

In [3]:
import h5py
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.figure(figsize=(10,10))

hist = pkl.load(open(directory + '/acgan-history.pkl', 'rb'))
losses = ['loss', 'generation_loss', 'auxiliary_loss']

for p in ['train', 'test']:
    for g in ['discriminator', 'generator']:
        hist[p][g] = pd.DataFrame(hist[p][g], columns=losses)

for p in ['train', 'test']:
    for g in ['discriminator', 'generator']:
        plt.plot(hist[p][g]['generation_loss'], label='{} ({})'.format(g, p))

# get the NE and show as an equilibrium point
plt.hlines(-np.log(0.5), 0, hist[p][g]['generation_loss'].shape[0], label='Nash Equilibrium')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel(r'$L_s$')

plt.show()

# for p in hist['privacy']:
#     print(p)

NameError: name 'directory' is not defined

In [ ]:
print(hist['privacy'][-1])

# Transfer Learning

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

acgan = __import__('ac_gan')

from glob import glob
from keras.models import load_model
from sklearn import linear_model

latent_size = 100


lr_clf = linear_model.LogisticRegression()
transfer_clf = RandomForestClassifier()

mean_scores = []
lr_scores = []
#directory = './output/p' + str(9) + '_8.0_0.0001_500_0.002_100/'
directory = './output/acgan_500_0.0002_100/'

for i in range(0, 500):
    gen_name = sorted(glob(directory + 'params_generator*'))[i]
    print(gen_name)
    g = load_model(gen_name)

    generate_count = training_size

    noise = np.random.uniform(-1, 1, (generate_count, latent_size))
    sampled_labels = np.random.randint(0, 2, generate_count)
    generated_images = g.predict([noise, sampled_labels.reshape((-1, 1))], verbose=0)

#     print(generated_images.shape)
    gen_X_train = np.reshape(generated_images, (training_size, 3, 12))

#     print(np.max(gen_X_train))
#     print(np.min(gen_X_train))

    #for i in range(gen_X_train.shape[0]):
    #    gen_X_train[i][0] = gen_X_train[i][0] 
    #    gen_X_train[i][1] = gen_X_train[i][1] 
    #    gen_X_train[i][2] = gen_X_train[i][2]

    gen_X_train = gen_X_train.astype(int)
    gen_X_train = gen_X_train.clip(min=0)
#     print(gen_X_train[:2])
#     print(gen_X_train.shape)

    gen_X_train = gen_X_train.reshape(generate_count, -1)
#     print(gen_X_train.shape)

    gen_y_train = sampled_labels

    #unique, counts = np.unique(y_train, return_counts=True)
#     print('real' + str(dict(zip(unique, counts))))

    #unique, counts = np.unique(gen_y_train, return_counts=True)
#     print('gen' + str(dict(zip(unique, counts))))

#     print(gen_X_train.shape)
#     print(gen_y_train.shape)

#from sklearn.model_selection import cross_val_score

#transfer_clf = RandomForestClassifier()
#scores = cross_val_score(transfer_clf, gen_X_train, gen_y_train, cv=5)
#print(scores)

#score_list = []
#for i in range(5):
#    gen_y_pred = transfer_clf.fit(gen_X_train, gen_y_train).predict(X_test)
#    score_list.append(accuracy_score(y_test, gen_y_pred))
#print(score_list)
#print(np.mean(score_list))
#mean_scores.append(np.mean(score_list))

    mean_scores.append(accuracy_score(y_test, transfer_clf.fit(gen_X_train, gen_y_train).predict(X_test)))
    lr_scores.append(accuracy_score(y_test, lr_clf.fit(gen_X_train, gen_y_train).predict(X_test)))
pkl.dump({'rf': mean_scores, 'lr': lr_scores}, open(directory + 'epoch_scores.p', 'wb'))

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.
./output/acgan_500_0.0002_100/params_generator_epoch_000.h5
./output/acgan_500_0.0002_100/params_generator_epoch_001.h5
./output/acgan_500_0.0002_100/params_generator_epoch_002.h5
./output/acgan_500_0.0002_100/params_generator_epoch_003.h5
./output/acgan_500_0.0002_100/params_generator_epoch_004.h5
./output/acgan_500_0.0002_100/params_generator_epoch_005.h5
./output/acgan_500_0.0002_100/params_generator_epoch_006.h5
./output/acgan_500_0.0002_100/params_generator_epoch_007.h5
./output/acgan_500_0.0002_100/params_generator_epoch_008.h5
./output/acgan_500_0.0002_100/params_generator_epoch_009.h5
./output/acgan_500_0.0002_100/params_generator_epoch_010.h5
./output/acgan_500_0.0002_100/params_generator_epoch_011.h5
./output/acgan_500_0.0002_100/params_generator_epoch_012.h5
./output/acgan_500_0.0002_100/params_generator_epoch_013.h5
./output/acgan_500_0.0002_100/params_generator_epoch_014.h5
./output/acgan_500_0.0002_100

In [ ]:
print(lr_scores)
print(max(lr_scores))
print(np.argmax(lr_scores))
print(lr_scores[np.argmax(lr_scores)])

In [ ]:
print(mean_scores)
print(max(mean_scores))
print(np.argmax(mean_scores))
print(mean_scores[np.argmax(mean_scores)])

In [ ]:
gen_name = sorted(glob(directory + 'params_generator*'))[np.argmax(lr_scores)]
print(gen_name)
g = load_model(gen_name)

generate_count = training_size

noise = np.random.uniform(-1, 1, (generate_count, latent_size))
sampled_labels = np.random.randint(0, 2, generate_count)
generated_images = g.predict([noise, sampled_labels.reshape((-1, 1))], verbose=0)

#     print(generated_images.shape)
gen_X_train = np.reshape(generated_images, (training_size, 3, 12))

#     print(np.max(gen_X_train))
#     print(np.min(gen_X_train))

for i in range(gen_X_train.shape[0]):
    gen_X_train[i][0] = gen_X_train[i][0] 
    gen_X_train[i][1] = gen_X_train[i][1] 
    gen_X_train[i][2] = gen_X_train[i][2]

gen_X_train = gen_X_train.astype(int)
gen_X_train = gen_X_train.clip(min=0)
#     print(gen_X_train[:2])
#     print(gen_X_train.shape)

gen_X_train = gen_X_train.reshape(generate_count, -1)
#     print(gen_X_train.shape)

gen_y_train = sampled_labels

unique, counts = np.unique(y_train, return_counts=True)
#     print('real' + str(dict(zip(unique, counts))))

unique, counts = np.unique(gen_y_train, return_counts=True)
#     print('gen' + str(dict(zip(unique, counts))))

#     print(gen_X_train.shape)
#     print(gen_y_train.shape)

from sklearn.model_selection import cross_val_score

transfer_clf = RandomForestClassifier()
scores = cross_val_score(transfer_clf, gen_X_train, gen_y_train, cv=5)
print(scores)

score_list = []
for i in range(5):
    gen_y_pred = transfer_clf.fit(gen_X_train, gen_y_train).predict(X_test)
    score_list.append(accuracy_score(y_test, gen_y_pred))
print(score_list)
print(np.mean(score_list))

# PCA plot

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X_pca = np.concatenate((gen_X_train, X_train))
print(X_pca.shape)

y = np.array([1] * generate_count + [0] * training_size)
pca = PCA(n_components=2)
X_r = pca.fit(X_pca).transform(X_pca)

target_names = ['Fake', 'Real']
colors = ['navy', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)

plt.title('PCA of Real vs. Fake')

print(pca.explained_variance_ratio_)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

plot_count = 500
X_pca = np.concatenate((gen_X_train[:plot_count], X_train[:plot_count]))
print(X_pca.shape)

y = np.array([1] * plot_count + [0] * plot_count)

pca = PCA(n_components=2)
X_r = pca.fit(X_pca).transform(X_pca)

target_names = ['Fake', 'Real']
colors = ['navy', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)

plt.title('PCA of Real vs. Fake')

# Real vs. Fake

In [ ]:
clf =  RandomForestClassifier()

X_comb = np.concatenate((gen_X_train, X_train))
y = np.array([1] * generate_count + [0] * training_size)

print(X_comb.shape, y.shape)
scores = cross_val_score(clf, X_comb, y, cv=5)
print(scores)

from sklearn.neighbors import KNeighborsClassifier
nn_clf = KNeighborsClassifier(n_neighbors=9)

scores = cross_val_score(nn_clf, X_comb, y, cv=5)
print(scores)